<a href="https://colab.research.google.com/github/heghiw/lakovaci-linka/blob/main/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Remove the existing directory
!rm -rf lakovaci-linka

# Clone the repository again
!git clone https://github.com/heghiw/lakovaci-linka.git

# Navigate to the directory
%cd lakovaci-linka


Cloning into 'lakovaci-linka'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 31 (delta 10), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 9.04 MiB | 24.88 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/lakovaci-linka/lakovaci-linka


data prep

In [13]:
import pandas as pd

# Define the path to the Excel file
file_path = '/content/lakovaci-linka/data1.xlsx'

# Read the sheet 'linka' into a DataFrame
linka_df = pd.read_excel(file_path, sheet_name='linka')

# Read the sheet 'recept' into a DataFrame
recept_df = pd.read_excel(file_path, sheet_name='recept')

print("Linka:")
print(linka_df)
print("\nRecepty:")
print(recept_df)



Linka:
             typ vany  id_vany   pozice_x_rel  pozice_x_cum
0               vstup         0           0.0           0.0
1        Teplý oplach         1        2300.0        2300.0
2             Postřik         2        1900.0        4200.0
3    Ponor odm železo         3        1900.0        6100.0
4    Ponor odm pozink         4        1800.0        7900.0
5            oplach 1         5        1800.0        9700.0
6            oplach 2         6        1800.0       11500.0
7       Moření železo         7        1800.0       13300.0
8       Moření pozink         8        1800.0       15100.0
9      Oplach moř žel         9        1800.0       16900.0
10  Oplach moř pozink        10        1800.0       18700.0
11    oplach společný        11        1800.0       20500.0
12           aktivace        12        1800.0       22300.0
13             fosfát        13        1800.0       24100.0
14             oplach        14        1800.0       25900.0
15            Oplach         21  

manipulator and vozik characteristics

In [14]:
linka_df.columns = linka_df.columns.str.strip()
recept_df.columns = recept_df.columns.str.strip()
print(linka_df.columns)
print(recept_df.columns)

Index(['typ vany', 'id_vany', 'pozice_x_rel', 'pozice_x_cum'], dtype='object')
Index(['tech', 'id_vany', 'poradi_operace', 'cas_min', 'cas_max', 'cas_opt',
       'okap', 'okap_cas'],
      dtype='object')


simulace


vizualizace


In [15]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 53.7 MB/s eta 0:00:00


ver5


In [22]:
import pandas as pd
import numpy as np


# -------------------------------
# User Input: Number of Manipulators
# -------------------------------
num_manipulators = 6  # User-specified number

# -------------------------------
# Manipulator Characteristics
# -------------------------------
manipulator_characteristics = {
    "okap_point": 2000,         # mm: height for okap (drip) during upward movement
    "highest_point": 2750,       # mm: maximum vertical height
    "deceleration_point": 500,   # mm: point where downward speed is reduced near the bath surface
    "speed_after_ponoreni": 8,   # m/min: slower speed used near the bath surface
    "speed_going_up": 15,        # m/min: upward speed
    "speed_going_down": 12,      # m/min: downward speed before deceleration
    "speed_left_right": 35       # m/min: horizontal speed
}

# -------------------------------
# Helper Functions & Movement Times
# -------------------------------
def m_per_sec(m_per_min):
    return m_per_min / 60.0

# Convert speeds to m/s:
speed_going_up = m_per_sec(manipulator_characteristics["speed_going_up"])
speed_going_down = m_per_sec(manipulator_characteristics["speed_going_down"])
speed_near_bath = m_per_sec(manipulator_characteristics["speed_after_ponoreni"])
speed_horizontal = m_per_sec(manipulator_characteristics["speed_left_right"])

# Compute vertical up time (from bath surface 0 m to highest_point)
highest_point_m = manipulator_characteristics["highest_point"] / 1000.0
vertical_up_time = highest_point_m / speed_going_up

# Compute vertical down time (using deceleration near bath)
decel_point_m = manipulator_characteristics["deceleration_point"] / 1000.0
time_down_phase1 = (highest_point_m - decel_point_m) / speed_going_down
time_down_phase2 = decel_point_m / speed_near_bath
vertical_down_time = time_down_phase1 + time_down_phase2

def compute_horizontal_time(x_from_mm, x_to_mm, speed_horizontal):
    distance_m = abs(x_to_mm - x_from_mm) / 1000.0
    return distance_m / speed_horizontal

# -------------------------------
# Cycle Settings & Product Entry
# -------------------------------
max_cycle_length = 300.0  # Maximum allowed cycle time (seconds)
# New products can enter as soon as the entry (first) bath is empty.
# Therefore, the cycle for the entry group is determined solely by its phases.

# -------------------------------
# Build Dwell Time Mapping from Technology Recipe
# -------------------------------
tech_dwell = {}
for idx, row in recept_df.iterrows():
    tech_dwell[row['id_vany']] = row['cas_opt'] + (row['okap_cas'] if row['okap'] else 0)

# -------------------------------
# Build Full Transitions for the Physical Line
# -------------------------------
physical_baths = linka_df_valid['id_vany'].tolist()
positions = linka_df_valid['pozice_x_cum'].tolist()

transitions = []
for i in range(len(physical_baths) - 1):
    from_bath = physical_baths[i]
    to_bath = physical_baths[i+1]
    pos_from = positions[i]
    pos_to = positions[i+1]

    horizontal_time = compute_horizontal_time(pos_from, pos_to, speed_horizontal)
    move_time = vertical_up_time + horizontal_time + vertical_down_time
    dwell_time = tech_dwell.get(from_bath, 0)  # if not used by tech, dwell = 0

    transitions.append({
        'from_bath': from_bath,
        'to_bath': to_bath,
        'pos_from': pos_from,
        'pos_to': pos_to,
        'horizontal_time': horizontal_time,
        'move_time': move_time,
        'dwell_time': dwell_time,
        'total_transition_time': move_time + dwell_time
    })

# -------------------------------
# Partition Transitions Among Manipulators
# -------------------------------
T = len(transitions)
if num_manipulators > T:
    print("Warning: Number of manipulators exceeds number of transitions; some will be idle.")
    num_manipulators = T

groups = []
start_idx = 0
base_size = T // num_manipulators
remainder = T % num_manipulators
for i in range(num_manipulators):
    size = base_size + (1 if i < remainder else 0)
    groups.append(transitions[start_idx:start_idx+size])
    start_idx += size

# -------------------------------
# Build Detailed Schedule for Each Manipulator Group
# -------------------------------
manipulator_schedules = []
for m, group in enumerate(groups):
    # For group 0 (entry), no extra dwell is added.
    if m == 0:
        extra_dwell = 0
    else:
        group_time = sum(tr['total_transition_time'] for tr in group)
        extra_dwell = max_cycle_length - group_time if group_time < max_cycle_length else 0
    phases = []
    current_time = 0
    for tr in group:
        phase = {}
        phase['vertical_up_start'] = current_time
        phase['vertical_up_end'] = current_time + vertical_up_time
        current_time += vertical_up_time

        phase['horizontal_start'] = current_time
        phase['horizontal_end'] = current_time + tr['horizontal_time']
        current_time += tr['horizontal_time']

        phase['vertical_down_start'] = current_time
        phase['vertical_down_end'] = current_time + vertical_down_time
        current_time += vertical_down_time

        phase['dwell_start'] = current_time
        phase['dwell_end'] = current_time + tr['dwell_time']
        current_time += tr['dwell_time']

        phase['from_bath'] = tr['from_bath']
        phase['to_bath'] = tr['to_bath']
        phases.append(phase)

    extra_phase = {'extra_dwell_start': current_time, 'extra_dwell_end': current_time + extra_dwell}
    current_time += extra_dwell
    start_offset = (max_cycle_length / num_manipulators) * m
    group_from = group[0]['from_bath']
    group_to = group[-1]['to_bath']
    pos_from = linka_df_valid.loc[linka_df_valid['id_vany'] == group_from, 'pozice_x_cum'].values[0]
    pos_to = linka_df_valid.loc[linka_df_valid['id_vany'] == group_to, 'pozice_x_cum'].values[0]

    manipulator_schedules.append({
        'manipulator_id': m,
        'assigned_transitions': group,
        'phases': phases,
        'extra_dwell': extra_phase,
        'total_cycle_time': current_time,
        'start_offset': start_offset,
        'physical_range': (group_from, group_to, pos_from, pos_to)
    })

# -------------------------------
# Create Summary Tables
# -------------------------------

# Summary table for each manipulator group
summary_rows = []
for sched in manipulator_schedules:
    rng = sched['physical_range']
    summary_rows.append({
        "Manipulator": sched['manipulator_id'],
        "Start Offset (s)": round(sched['start_offset'], 2),
        "Physical Range": f"{rng[0]} (pos: {rng[2]} mm) -> {rng[1]} (pos: {rng[3]} mm)",
        "Cycle Time (s)": round(sched['total_cycle_time'], 2)
    })
summary_df = pd.DataFrame(summary_rows)

# Detailed phase table: one row per phase per manipulator
phase_rows = []
for sched in manipulator_schedules:
    for idx, phase in enumerate(sched['phases']):
        phase_rows.append({
            "Manipulator": sched['manipulator_id'],
            "Transition": idx,
            "From Bath": phase['from_bath'],
            "To Bath": phase['to_bath'],
            "Vertical Up": f"{phase['vertical_up_start']:.2f}-{phase['vertical_up_end']:.2f}",
            "Horizontal": f"{phase['horizontal_start']:.2f}-{phase['horizontal_end']:.2f}",
            "Vertical Down": f"{phase['vertical_down_start']:.2f}-{phase['vertical_down_end']:.2f}",
            "Dwell": f"{phase['dwell_start']:.2f}-{phase['dwell_end']:.2f}"
        })
detailed_phase_df = pd.DataFrame(phase_rows)

# Full physical ranges table (from all baths)
range_rows = []
for i in range(len(linka_df_valid) - 1):
    range_rows.append({
        "Range Index": i,
        "From Bath": linka_df_valid.loc[i, 'id_vany'],
        "To Bath": linka_df_valid.loc[i+1, 'id_vany'],
        "From Position (mm)": linka_df_valid.loc[i, 'pozice_x_cum'],
        "To Position (mm)": linka_df_valid.loc[i+1, 'pozice_x_cum']
    })
ranges_df = pd.DataFrame(range_rows)

# -------------------------------
# Product Entry Times
# -------------------------------
# The entry group (group 0) finishes its cycle as soon as the first bath is empty.
entry_cycle_time = manipulator_schedules[0]['total_cycle_time']
num_cycles = 5
product_entry_times = [round(i * entry_cycle_time, 2) for i in range(num_cycles+1)]
entry_df = pd.DataFrame({"Cycle": list(range(num_cycles+1)), "Product Entry Time (s)": product_entry_times})

# -------------------------------
# Print Results in Table Form
# -------------------------------
print("Summary Schedule for Each Manipulator:")
print(summary_df.to_string(index=False))

print("\nDetailed Phase Schedule:")
print(detailed_phase_df.to_string(index=False))

print("\nFull Physical Manipulator Ranges (from all baths):")
print(ranges_df.to_string(index=False))

print("\nProduct Entry Times (new product enters as soon as bath 0 is empty):")
print(entry_df.to_string(index=False))


Summary Schedule for Each Manipulator:
 Manipulator  Start Offset (s)                                Physical Range  Cycle Time (s)
           0               0.0         0 (pos: 0.0 mm) -> 4 (pos: 7900.0 mm)          657.54
           1              50.0     4 (pos: 7900.0 mm) -> 8 (pos: 15100.0 mm)          766.34
           2             100.0   8 (pos: 15100.0 mm) -> 12 (pos: 22300.0 mm)          300.00
           3             150.0  12 (pos: 22300.0 mm) -> 15 (pos: 29500.0 mm)          300.00
           4             200.0  15 (pos: 29500.0 mm) -> 18 (pos: 35900.0 mm)          300.00
           5             250.0 18 (pos: 35900.0 mm) -> 100 (pos: 41800.0 mm)          300.00

Detailed Phase Schedule:
 Manipulator  Transition  From Bath  To Bath   Vertical Up    Horizontal Vertical Down         Dwell
           0           0          0        1    0.00-11.00   11.00-14.94   14.94-29.94   29.94-29.94
           0           1          1        2   29.94-40.94   40.94-44.20   44.20-5